# مکانیزم‌های توجه و مدل‌های ترانسفورمر

یکی از مشکلات اصلی شبکه‌های بازگشتی این است که تمام کلمات در یک دنباله تأثیر یکسانی بر نتیجه دارند. این موضوع باعث عملکرد زیر حد مطلوب در مدل‌های استاندارد LSTM برای وظایف دنباله به دنباله، مانند شناسایی موجودیت‌های نام‌دار و ترجمه ماشینی می‌شود. در واقعیت، برخی کلمات در دنباله ورودی تأثیر بیشتری بر خروجی‌های دنباله‌ای نسبت به دیگر کلمات دارند.

به مدل دنباله به دنباله، مانند ترجمه ماشینی، توجه کنید. این مدل توسط دو شبکه بازگشتی پیاده‌سازی می‌شود، که در آن یک شبکه (**رمزگذار**) دنباله ورودی را به حالت پنهان تبدیل می‌کند و شبکه دیگر (**رمزگشا**) این حالت پنهان را به نتیجه ترجمه‌شده باز می‌گرداند. مشکل این روش این است که حالت نهایی شبکه به سختی می‌تواند ابتدای جمله را به خاطر بسپارد، که این موضوع باعث کاهش کیفیت مدل در جملات طولانی می‌شود.

**مکانیزم‌های توجه** راهی برای وزن‌دهی تأثیر متنی هر بردار ورودی بر هر پیش‌بینی خروجی RNN فراهم می‌کنند. این مکانیزم با ایجاد میان‌برهایی بین حالت‌های میانی RNN ورودی و RNN خروجی پیاده‌سازی می‌شود. به این ترتیب، هنگام تولید نماد خروجی $y_t$، تمام حالت‌های پنهان ورودی $h_i$ را با ضرایب وزنی مختلف $\alpha_{t,i}$ در نظر می‌گیریم.

![تصویری که مدل رمزگذار/رمزگشا با لایه توجه افزایشی را نشان می‌دهد](../../../../../lessons/5-NLP/18-Transformers/images/encoder-decoder-attention.png)
*مدل رمزگذار-رمزگشا با مکانیزم توجه افزایشی در [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf)، نقل شده از [این پست وبلاگ](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

ماتریس توجه $\{\alpha_{i,j}\}$ نشان‌دهنده درجه‌ای است که کلمات خاص ورودی در تولید یک کلمه مشخص در دنباله خروجی نقش دارند. در زیر نمونه‌ای از چنین ماتریسی آورده شده است:

![تصویری که یک نمونه هم‌ترازی پیدا شده توسط RNNsearch-50 را نشان می‌دهد، گرفته شده از Bahdanau - arviz.org](../../../../../lessons/5-NLP/18-Transformers/images/bahdanau-fig3.png)

*تصویر گرفته شده از [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (شکل ۳)*

مکانیزم‌های توجه مسئول بسیاری از دستاوردهای فعلی یا نزدیک به فعلی در پردازش زبان طبیعی هستند. با این حال، افزودن توجه به طور قابل توجهی تعداد پارامترهای مدل را افزایش می‌دهد که منجر به مشکلات مقیاس‌پذیری در RNN‌ها می‌شود. یکی از محدودیت‌های کلیدی در مقیاس‌پذیری RNN‌ها این است که ماهیت بازگشتی مدل‌ها باعث می‌شود آموزش دسته‌ای و موازی‌سازی چالش‌برانگیز شود. در RNN، هر عنصر از دنباله باید به ترتیب پردازش شود، که به این معناست که نمی‌توان آن را به راحتی موازی‌سازی کرد.

پذیرش مکانیزم‌های توجه همراه با این محدودیت منجر به ایجاد مدل‌های ترانسفورمر شد که اکنون به عنوان پیشرفته‌ترین مدل‌ها شناخته می‌شوند و از BERT تا OpenGPT3 مورد استفاده قرار می‌گیرند.

## مدل‌های ترانسفورمر

به جای انتقال زمینه هر پیش‌بینی قبلی به مرحله ارزیابی بعدی، **مدل‌های ترانسفورمر** از **کدگذاری موقعیتی** و **توجه** برای درک زمینه ورودی در یک پنجره متن مشخص استفاده می‌کنند. تصویر زیر نشان می‌دهد که چگونه کدگذاری موقعیتی همراه با توجه می‌تواند زمینه را در یک پنجره مشخص درک کند.

![GIF متحرک که نشان می‌دهد ارزیابی‌ها در مدل‌های ترانسفورمر چگونه انجام می‌شوند.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

از آنجا که هر موقعیت ورودی به طور مستقل به هر موقعیت خروجی نگاشت می‌شود، ترانسفورمرها بهتر از RNN‌ها موازی‌سازی می‌شوند، که این امکان را فراهم می‌کند تا مدل‌های زبانی بزرگ‌تر و بیان‌گرتر ایجاد شوند. هر سر توجه می‌تواند برای یادگیری روابط مختلف بین کلمات استفاده شود که وظایف پردازش زبان طبیعی را بهبود می‌بخشد.

## ساخت مدل ترانسفورمر ساده

Keras شامل لایه ترانسفورمر داخلی نیست، اما می‌توانیم خودمان آن را بسازیم. همانند قبل، ما بر طبقه‌بندی متن مجموعه داده AG News تمرکز خواهیم کرد، اما شایان ذکر است که مدل‌های ترانسفورمر بهترین نتایج را در وظایف دشوارتر NLP نشان می‌دهند.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

لایه‌های جدید در Keras باید کلاس `Layer` را زیرکلاس کنند و متد `call` را پیاده‌سازی کنند. بیایید با لایه **جایگذاری موقعیتی** شروع کنیم. ما از [کدی از مستندات رسمی Keras](https://keras.io/examples/nlp/text_classification_with_transformer/) استفاده خواهیم کرد. فرض می‌کنیم که تمام دنباله‌های ورودی را به طول `maxlen` پر می‌کنیم.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

این لایه شامل دو لایه `Embedding` است: یکی برای جاسازی توکن‌ها (به شیوه‌ای که قبلاً بحث کردیم) و دیگری برای موقعیت توکن‌ها. موقعیت توکن‌ها به صورت یک دنباله از اعداد طبیعی از 0 تا `maxlen` با استفاده از `tf.range` ایجاد می‌شوند و سپس از طریق لایه جاسازی عبور داده می‌شوند. دو بردار جاسازی حاصل با هم جمع می‌شوند و نمایشی جاسازی‌شده موقعیتی از ورودی با شکل `maxlen`$\times$`embed_dim` تولید می‌کنند.

حالا بیایید بلوک ترانسفورمر را پیاده‌سازی کنیم. این بلوک خروجی لایه جاسازی تعریف‌شده قبلی را دریافت خواهد کرد:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

اکنون آماده‌ایم مدل کامل ترانسفورمر را تعریف کنیم:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## مدل‌های ترنسفورمر BERT

**BERT** (نمایش‌های رمزگذار دوطرفه از ترنسفورمرها) یک شبکه ترنسفورمر چندلایه بسیار بزرگ است که برای *BERT-base* شامل ۱۲ لایه و برای *BERT-large* شامل ۲۴ لایه می‌باشد. این مدل ابتدا با استفاده از یک مجموعه داده متنی بزرگ (ویکی‌پدیا + کتاب‌ها) و با روش آموزش بدون نظارت (پیش‌بینی کلمات ماسک‌شده در یک جمله) پیش‌آموزش داده می‌شود. در طول پیش‌آموزش، مدل سطح قابل توجهی از درک زبان را جذب می‌کند که می‌توان از آن در ترکیب با مجموعه داده‌های دیگر از طریق تنظیم دقیق استفاده کرد. این فرآیند **یادگیری انتقالی** نامیده می‌شود.

![تصویر از http://jalammar.github.io/illustrated-bert/](../../../../../lessons/5-NLP/18-Transformers/images/jalammarBERT-language-modeling-masked-lm.png)

معماری‌های مختلفی از ترنسفورمرها وجود دارند، از جمله BERT، DistilBERT، BigBird، OpenGPT3 و موارد دیگر که می‌توان آن‌ها را تنظیم دقیق کرد.

بیایید ببینیم چگونه می‌توان از مدل پیش‌آموزش‌داده‌شده BERT برای حل مسئله سنتی طبقه‌بندی توالی استفاده کرد. ما ایده و بخشی از کد را از [مستندات رسمی](https://www.tensorflow.org/text/tutorials/classify_text_with_bert) قرض می‌گیریم.

برای بارگذاری مدل‌های پیش‌آموزش‌داده‌شده، از **Tensorflow hub** استفاده خواهیم کرد. ابتدا، بیایید وکتورایزر مخصوص BERT را بارگذاری کنیم:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

مهم است که از همان وکتورایزر استفاده کنید که شبکه اصلی با آن آموزش داده شده است. همچنین، وکتورایزر BERT سه مؤلفه بازمی‌گرداند:
* `input_word_ids`، که یک دنباله از شماره‌های توکن برای جمله ورودی است
* `input_mask`، که نشان می‌دهد کدام بخش از دنباله شامل ورودی واقعی است و کدام بخش پدینگ است. این مشابه ماسکی است که توسط لایه `Masking` تولید می‌شود
* `input_type_ids` برای وظایف مدل‌سازی زبان استفاده می‌شود و امکان مشخص کردن دو جمله ورودی در یک دنباله را فراهم می‌کند.

سپس، می‌توانیم استخراج‌کننده ویژگی BERT را ایجاد کنیم:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


لایه BERT چندین نتیجه مفید ارائه می‌دهد:
* `pooled_output` نتیجه میانگین‌گیری تمام توکن‌ها در دنباله است. می‌توانید آن را به‌عنوان یک جاسازی معنایی هوشمند برای کل شبکه در نظر بگیرید. این معادل خروجی لایه `GlobalAveragePooling1D` در مدل قبلی ما است.
* `sequence_output` خروجی آخرین لایه ترانسفورمر است (مطابق با خروجی `TransformerBlock` در مدل بالا).
* `encoder_outputs` خروجی‌های تمام لایه‌های ترانسفورمر هستند. از آنجا که ما مدل BERT چهار لایه بارگذاری کرده‌ایم (همان‌طور که احتمالاً از نام آن که شامل `4_H` است حدس زده‌اید)، این مدل دارای چهار تنسور است. آخرین تنسور همان `sequence_output` است.

اکنون مدل طبقه‌بندی انتها به انتها را تعریف خواهیم کرد. از *تعریف مدل تابعی* استفاده خواهیم کرد، جایی که ورودی مدل را تعریف می‌کنیم و سپس مجموعه‌ای از عبارات برای محاسبه خروجی آن ارائه می‌دهیم. همچنین وزن‌های مدل BERT را غیرقابل آموزش قرار می‌دهیم و فقط طبقه‌بند نهایی را آموزش می‌دهیم:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


با وجود تعداد کم پارامترهای قابل آموزش، فرآیند نسبتاً کند است، زیرا استخراج‌کننده ویژگی‌های BERT از نظر محاسباتی سنگین است. به نظر می‌رسد که نتوانستیم دقت قابل قبولی به دست آوریم، یا به دلیل کمبود آموزش، یا کمبود پارامترهای مدل.

بیایید سعی کنیم وزن‌های BERT را از حالت فریز خارج کنیم و آن را نیز آموزش دهیم. این کار نیاز به نرخ یادگیری بسیار کوچک دارد و همچنین استراتژی آموزشی دقیق‌تری با **گرم کردن** و استفاده از بهینه‌ساز **AdamW** دارد. ما از بسته `tf-models-official` برای ایجاد بهینه‌ساز استفاده خواهیم کرد:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

همانطور که می‌بینید، فرآیند آموزش مدل به کندی پیش می‌رود - اما ممکن است بخواهید آزمایش کنید و مدل را برای چند دوره (۵-۱۰) آموزش دهید و ببینید آیا می‌توانید نتیجه بهتری نسبت به روش‌هایی که قبلاً استفاده کرده‌ایم، به دست آورید.

## کتابخانه Huggingface Transformers

یکی دیگر از روش‌های بسیار رایج (و کمی ساده‌تر) برای استفاده از مدل‌های Transformer، [بسته HuggingFace](https://github.com/huggingface/) است که بلوک‌های سازنده ساده‌ای برای وظایف مختلف پردازش زبان طبیعی (NLP) ارائه می‌دهد. این کتابخانه هم برای Tensorflow و هم برای PyTorch، که یکی دیگر از چارچوب‌های محبوب شبکه‌های عصبی است، در دسترس است.

> **توجه**: اگر علاقه‌ای به دیدن نحوه عملکرد کتابخانه Transformers ندارید - می‌توانید به انتهای این دفترچه یادداشت بروید، زیرا چیز متفاوتی نسبت به آنچه قبلاً انجام داده‌ایم نخواهید دید. ما همان مراحل آموزش مدل BERT را با استفاده از کتابخانه‌ای متفاوت و مدلی به‌مراتب بزرگ‌تر تکرار خواهیم کرد. بنابراین، این فرآیند شامل آموزش‌های نسبتاً طولانی است، و ممکن است بخواهید فقط کدها را مرور کنید.

بیایید ببینیم چگونه می‌توانیم مسئله خود را با استفاده از [Huggingface Transformers](http://huggingface.co) حل کنیم.


اولین کاری که باید انجام دهیم انتخاب مدلی است که قصد استفاده از آن را داریم. علاوه بر مدل‌های داخلی، Huggingface یک [مخزن آنلاین مدل‌ها](https://huggingface.co/models) دارد که در آن می‌توانید تعداد زیادی مدل از پیش آموزش‌دیده شده توسط جامعه را پیدا کنید. تمام این مدل‌ها تنها با ارائه نام مدل قابل بارگذاری و استفاده هستند. تمام فایل‌های باینری مورد نیاز برای مدل به صورت خودکار دانلود می‌شوند.

در برخی مواقع ممکن است نیاز داشته باشید مدل‌های خودتان را بارگذاری کنید. در این صورت می‌توانید دایرکتوری‌ای را مشخص کنید که شامل تمام فایل‌های مرتبط باشد، از جمله پارامترهای مربوط به tokenizer، فایل `config.json` با پارامترهای مدل، وزن‌های باینری و غیره.

از طریق نام مدل، می‌توانیم هم مدل و هم tokenizer را ایجاد کنیم. بیایید با یک tokenizer شروع کنیم:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

شیء `tokenizer` شامل تابع `encode` است که می‌توان آن را مستقیماً برای کدگذاری متن استفاده کرد:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

ما همچنین می‌توانیم از توکنایزر برای رمزگذاری یک دنباله به روشی مناسب برای ارسال به مدل استفاده کنیم، یعنی شامل فیلدهای `token_ids`، `input_mask` و غیره. ما همچنین می‌توانیم مشخص کنیم که می‌خواهیم تنسورهای Tensorflow را با ارائه آرگومان `return_tensors='tf'` دریافت کنیم:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

در اینجا از مدل از پیش آموزش‌دیده‌ی BERT به نام `bert-base-uncased` استفاده خواهیم کرد. *Uncased* نشان‌دهنده‌ی این است که مدل نسبت به حروف بزرگ و کوچک حساس نیست.

هنگام آموزش مدل، باید دنباله‌ی توکن‌شده را به عنوان ورودی ارائه دهیم، و به همین دلیل یک خط پردازش داده طراحی خواهیم کرد. از آنجا که `tokenizer.encode` یک تابع پایتون است، از همان رویکردی که در واحد قبلی استفاده کردیم بهره خواهیم برد و آن را با استفاده از `py_function` فراخوانی می‌کنیم:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

اکنون می‌توانیم مدل واقعی را با استفاده از بسته `BertForSequenceClassification` بارگذاری کنیم. این اطمینان حاصل می‌کند که مدل ما از قبل معماری مورد نیاز برای طبقه‌بندی، از جمله طبقه‌بند نهایی را دارد. شما پیام هشداری خواهید دید که بیان می‌کند وزن‌های طبقه‌بند نهایی مقداردهی اولیه نشده‌اند و مدل نیاز به پیش‌آموزش دارد - این کاملاً مشکلی ندارد، زیرا دقیقاً همان چیزی است که قصد انجام آن را داریم!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


همانطور که از `summary()` می‌بینید، مدل تقریباً ۱۱۰ میلیون پارامتر دارد! احتمالاً، اگر بخواهیم یک وظیفه طبقه‌بندی ساده روی مجموعه داده نسبتاً کوچک انجام دهیم، نمی‌خواهیم لایه پایه BERT را آموزش دهیم:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


اکنون آماده شروع آموزش هستیم!

> **توجه**: آموزش مدل کامل BERT می‌تواند بسیار زمان‌بر باشد! بنابراین، ما فقط آن را برای ۳۲ دسته اول آموزش خواهیم داد. این فقط برای نشان دادن نحوه تنظیم آموزش مدل است. اگر علاقه‌مند به امتحان آموزش کامل هستید - فقط پارامترهای `steps_per_epoch` و `validation_steps` را حذف کنید و آماده صبر کردن باشید!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


اگر تعداد تکرارها را افزایش دهید، زمان بیشتری منتظر بمانید و مدل را برای چندین دوره آموزش دهید، می‌توانید انتظار داشته باشید که طبقه‌بندی BERT بهترین دقت را به ما بدهد! دلیل این موضوع این است که BERT ساختار زبان را به‌خوبی درک کرده است و ما فقط نیاز داریم که طبقه‌بند نهایی را به‌صورت دقیق تنظیم کنیم. با این حال، چون BERT یک مدل بزرگ است، کل فرآیند آموزش زمان زیادی می‌برد و به قدرت محاسباتی قابل توجهی نیاز دارد! (GPU و ترجیحاً بیش از یک عدد).

> **توجه:** در مثال ما، از یکی از کوچک‌ترین مدل‌های BERT از پیش آموزش‌دیده استفاده کرده‌ایم. مدل‌های بزرگ‌تری وجود دارند که احتمالاً نتایج بهتری ارائه می‌دهند.


## نکات کلیدی

در این بخش، ما با معماری‌های مدل بسیار جدید مبتنی بر **ترانسفورمرها** آشنا شدیم. این مدل‌ها را برای وظیفه طبقه‌بندی متن به کار بردیم، اما به همین شکل، مدل‌های BERT می‌توانند برای استخراج موجودیت‌ها، پاسخ به سوالات و سایر وظایف پردازش زبان طبیعی استفاده شوند.

مدل‌های ترانسفورمر نمایانگر پیشرفته‌ترین روش‌ها در پردازش زبان طبیعی هستند و در بیشتر موارد باید اولین راه‌حلی باشند که هنگام پیاده‌سازی راه‌حل‌های سفارشی پردازش زبان طبیعی شروع به آزمایش با آن‌ها می‌کنید. با این حال، درک اصول پایه‌ای شبکه‌های عصبی بازگشتی که در این بخش مورد بحث قرار گرفتند، بسیار مهم است اگر قصد دارید مدل‌های عصبی پیشرفته بسازید.



---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، توصیه می‌شود از ترجمه حرفه‌ای انسانی استفاده کنید. ما مسئولیتی در قبال سوءتفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
